# TruthLens Modelling

In [1]:
!pip install lime

In [2]:
#imports
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import csv
import random
pd.set_option('display.max_colwidth', None)
from lime.lime_text import LimeTextExplainer

## Phase 1: Binary Classification

### Feature Extraction Using TF-IDF, n-grams and readability metrics

In [3]:
#load data
df = pd.read_csv('Data/phase1_final_clean.csv')
df = df.reset_index(drop=True)
print(df.head(3))

In [5]:
#TF-IDF feature extraction with n-grams
start_time = time.time()
#replace NaN values with an empty string to resolve NaN ValueError
df['content_lemma_nostop'] = df['content_lemma_nostop'].fillna('')
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
X_tfidf = vectorizer.fit_transform(df['content_lemma_nostop'])
#get pre-calculated readability features
readability_features = df[['word_count', 'sentence_count', 'flesch_reading_ease']].values
#standardise readability features
scaler = StandardScaler()
readability_scaled = scaler.fit_transform(readability_features)
#convert to a sparse matrix
readability_sparse = csr_matrix(readability_scaled)
#combine TF-IDF features with the readability metrics
X_combined = hstack([X_tfidf, readability_sparse])

y = df['label']
print("Feature extraction: {:.4f} seconds".format(time.time() - start_time))

Feature extraction: 371.5113 seconds


### Split dataset

In [6]:
#retain the indices as we need these for looking up explanations later
train_indices, test_indices = train_test_split(df.index, test_size=0.2, random_state=999)
#split X and y using the train/test indices
X_train = X_combined[train_indices]
X_test = X_combined[test_indices]
y_train = y.iloc[train_indices]
y_test = y.iloc[test_indices]

### Logistic Regression

In [8]:
start_time = time.time()
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Fit Logistic Regression model: {:.4f} seconds".format(time.time() - start_time))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Fit Logistic Regression model: 16.6642 seconds
Accuracy: 0.9419773508079908
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93      6931
           1       0.94      0.95      0.95      8787

    accuracy                           0.94     15718
   macro avg       0.94      0.94      0.94     15718
weighted avg       0.94      0.94      0.94     15718



### Random Forest

In [9]:
start_time = time.time()
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
print("Fit Random Forest model: {:.4f} seconds".format(time.time() - start_time))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("Random Forest Classification Report:\n", classification_report(y_test, rf_pred))

Fit Random Forest model: 618.2777 seconds
Random Forest Accuracy: 0.9505026084743606
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.94      6931
           1       0.95      0.96      0.96      8787

    accuracy                           0.95     15718
   macro avg       0.95      0.95      0.95     15718
weighted avg       0.95      0.95      0.95     15718



### Support Vector Machine (SVM)

In [34]:
start_time = time.time()
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
print("Fit SVM model: {:.4f} seconds".format(time.time() - start_time))
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("SVM Classification Report:\n", classification_report(y_test, svm_pred))

Fit SVM model: 4321.4944 seconds
SVM Accuracy: 0.9504389871484922
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      6931
           1       0.95      0.96      0.96      8787

    accuracy                           0.95     15718
   macro avg       0.95      0.95      0.95     15718
weighted avg       0.95      0.95      0.95     15718



### Tweak model

In [10]:
start_time = time.time()
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best parameters:", grid.best_params_)
print("Grid Search: {:.4f} seconds".format(time.time() - start_time))

Best parameters: {'C': 10}
Grid Search: 309.6480 seconds


In [11]:
# Instantiate the model with the best hyperparameter
final_model = LogisticRegression(C=10, max_iter=1000)

# Fit the final model on the training data
final_model.fit(X_train, y_train)

# Evaluate on the test set
final_predictions = final_model.predict(X_test)
print("Final Model Accuracy:", accuracy_score(y_test, final_predictions))
print("Final Model Classification Report:\n", classification_report(y_test, final_predictions))

Final Model Accuracy: 0.9487212113500445
Final Model Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.94      6931
           1       0.95      0.96      0.95      8787

    accuracy                           0.95     15718
   macro avg       0.95      0.95      0.95     15718
weighted avg       0.95      0.95      0.95     15718



### Explain prediction

In [30]:
def explain_prediction(row, model, vectorizer, scaler, top_n=5):
    """
        Explains the prediction of the model by showing the most influential features (both TF-IDF and readability metrics) for the prediction.

        Parameters:
        ----------
        row : pandas.Series
            A row of the dataframe containing:
              - 'text': the preprocessed text (e.g. from 'content_lemma_nostop')
              - 'word_count': the document's word count
              - 'sentence_count': the number of sentences
              - 'flasch_reading_ease': the readability score
        model : object
            The trained machine learning model.
        vectorizer : object
            The TF-IDF vectorizer used to transform the text.
        scaler : object
            The scaler used for the readability metrics during training.
        top_n : int
            The number of features to display based on their contribution.

        Returns:
        -------
        dict
            A dictionary with the prediction ('label') and the top contributing features ('features').
    """
    #make sure ther eis text in the row
    row['text'] = row['text'] if pd.notnull(row['text']) else ''
    #transform the text using the vectorizer (this gets the first 5000 features)
    tfidf_text = vectorizer.transform([row['text']])
    #get the readability features from the row
    readability_values = np.array([[row['word_count'], row['sentence_count'], row['flesch_reading_ease']]])
    #scale the readability metrics using the same scaler as during training.
    readability_scaled = scaler.transform(readability_values)
    #convert the scaled features into a sparse matrix
    readability_sparse = csr_matrix(readability_scaled)
    #combine TF-IDF features and readability features
    full_features = hstack([tfidf_text, readability_sparse])
    # Predict the label.
    prediction = model.predict(full_features)[0]
    
    #get contribution for each feature
    contributions = full_features.toarray()[0]* model.coef_[0]
    #get indices of features with highest absolute contributions
    top_indices = np.argsort(np.abs(contributions))[-top_n:]
    #get feature names
    tfidf_feature_names = vectorizer.get_feature_names_out()
    readability_feature_names = ['word_count', 'sentence_count', 'flesch_reading_ease']
    all_feature_names = list(tfidf_feature_names) + readability_feature_names
    top_features = [all_feature_names[i] for i in top_indices]

    return {
        "label": prediction,
        "features": top_features
    }

# Create a DataFrame for test data
test_df = pd.DataFrame({
    'text': df.loc[test_indices, 'content_lemma_nostop'].reset_index(drop=True),
    'word_count': df.loc[test_indices, 'word_count'].reset_index(drop=True),
    'sentence_count': df.loc[test_indices, 'sentence_count'].reset_index(drop=True),
    'flesch_reading_ease': df.loc[test_indices, 'flesch_reading_ease'].reset_index(drop=True),
    'true_label': y_test.reset_index(drop=True),
    'predicted_label': y_pred
})

# Row predicted as Real (0)
real_example = test_df[test_df['predicted_label'] == 0].iloc[0]
#print(real_example)

# Row predicted as Fake (1)
fake_example = test_df[test_df['predicted_label'] == 1].iloc[0]

In [32]:
real_explanation = explain_prediction(real_example, final_model, vectorizer, scaler)
fake_explanation = explain_prediction(fake_example, final_model, vectorizer, scaler)

print("Real Example Prediction:")
print("Text:", real_example['text'])
print("Predicted Label:", real_explanation['label'])
print("Top Features:", real_explanation['features'])

print("\nFake Example Prediction:")
print("Text:", fake_example['text'])
print("Predicted Label:", fake_explanation['label'])
print("Top Features:", fake_explanation['features'])


Real Example Prediction:
Text: among senseless beating inflict reality presidential debate monday night discussion new york city tactic donald j trump attribute nonexistent increase murder action never happen namely end practice variously judge judge current mayor multilayered fiction murder decline judge end neither current mayor fact police department begin drastically curtail use 2012 administration mayor michael r bloomberg independent well document lightly notice july 9 2012 editorial new york post warn reduction use would lead blood street way blood run street less blood murder 32 percent since 2011 last year old era drop 352 homicide 2015 515 2011 period stop 97 percent say j peter donald spokesman department debate hillary clinton democratic nominee allude number mr trump republican opponent start shout current mayor crime continue drop include murder say — — wrong mr trump interject wrong mrs clinton say murders right mr trump say check right mr trump say let check google sear

In [ ]:
def predict_proba(texts):
    """
    This function takes a list of raw text strings, vectorizes them using the
    pre-fitted TF-IDF vectorizer, and returns the probability predictions
    from the trained model.
    """
    return model.predict_proba(vectorizer.transform(texts))

# Create a LIME text explainer
explainer = LimeTextExplainer(class_names=['true', 'fake'])

# Choose an instance from the test set by its index
idx = 0  # you can adjust this index as needed
raw_text = df.loc[test_indices].iloc[idx]['content']

# Use the custom predict_proba function in LIME
exp = explainer.explain_instance(raw_text, predict_proba, num_features=10)
exp.show_in_notebook(text=raw_text)

### Validate model
Extra data from https://www.kaggle.com/datasets/stevenpeutz/misinformation-fake-news-text-dataset-79k

In [33]:
#load in validation dataset
external_df = pd.read_csv('Data/extra_data_final_clean.csv')
external_df = external_df.dropna(subset=['content_lemma_nostop']).reset_index(drop=True)
external_text = external_df['content_lemma_nostop']
external_labels = external_df['label']

# Transform the external data using the already fitted vectorizer (or pipeline)
X_external = vectorizer.transform(external_text)

# Predict using the trained model
external_predictions_lr = final_model.predict(X_external)

# Evaluate the performance on the external dataset
print("External Dataset Classification Report:\n", classification_report(external_labels, external_predictions_lr))


ValueError: X has 5000 features, but LogisticRegression is expecting 5003 features as input.

## Phase 2 

In [23]:
df2 = pd.read_csv('Data/custom_cleaned.csv')

import pandas as pd
import numpy as np
import gensim
from gensim import corpora
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from textblob import TextBlob
import spacy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load a spaCy model for NER
nlp = spacy.load('en_core_web_sm')

# Assume df2 is already loaded and cleaned:
# df2 = pd.read_csv('your_cleaned_df2.csv')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_multiclass, y_multiclass, test_size=0.2, random_state=42)
# Train a classifier on the training data
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the held-out test set
predictions = clf.predict(X_test)
print("Classification Report on Test Set:\n", classification_report(y_test, predictions))

###########################
# 1. Topic Modeling (LDA)
###########################
# Preprocess text for LDA: tokenize and remove stopwords
stop_words = set(stopwords.words('english'))
def preprocess(text):
    tokens = [token for token in gensim.utils.simple_preprocess(text) if token not in stop_words]
    return tokens

df2['tokens'] = df2['content'].apply(preprocess)

# Create a dictionary and corpus for LDA
dictionary = corpora.Dictionary(df2['tokens'])
corpus = [dictionary.doc2bow(text) for text in df2['tokens']]

# Train LDA model (e.g., 5 topics)
lda_model = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

# Function to extract topic distribution for a document
def get_topic_distribution(text):
    bow = dictionary.doc2bow(preprocess(text))
    # Get topic probabilities for all topics (ensure all topics are returned)
    topic_dist = lda_model.get_document_topics(bow, minimum_probability=0.0)
    # Return a list of probabilities ordered by topic index
    return [prob for topic, prob in sorted(topic_dist, key=lambda x: x[0])]

df2['topic_dist'] = df2['content'].apply(get_topic_distribution)

###############################
# 2. Sentiment Analysis
###############################
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

df2[['polarity', 'subjectivity']] = df2['content'].apply(lambda x: pd.Series(get_sentiment(x)))

#######################################
# 3. Named Entity Recognition (NER)
#######################################
def get_entity_counts(text):
    doc = nlp(text)
    counts = {'PERSON': 0, 'ORG': 0, 'GPE': 0}
    for ent in doc.ents:
        if ent.label_ in counts:
            counts[ent.label_] += 1
    return counts

df2['entity_counts'] = df2['content'].apply(get_entity_counts)
df2['person_count'] = df2['entity_counts'].apply(lambda x: x['PERSON'])
df2['org_count'] = df2['entity_counts'].apply(lambda x: x['ORG'])
df2['gpe_count'] = df2['entity_counts'].apply(lambda x: x['GPE'])

########################################
# 4. Domain-Specific Keyword Counts
########################################
# Define a dictionary for domain-specific keywords
domain_keywords = {
    'politics': ['election', 'government', 'senate', 'congress'],
    'health': ['vaccine', 'covid', 'pandemic', 'healthcare'],
    'finance': ['stock', 'market', 'economy', 'trade']
}

def count_domain_keywords(text):
    counts = {}
    text_lower = text.lower()
    for category, keywords in domain_keywords.items():
        count = sum(text_lower.count(keyword) for keyword in keywords)
        counts[category] = count
    return counts

df2['domain_counts'] = df2['content'].apply(count_domain_keywords)
# Expand domain keyword counts into separate columns
domain_df = df2['domain_counts'].apply(pd.Series)
df2 = pd.concat([df2, domain_df], axis=1)

###############################################
# 5. Combine Features into a Single Feature Vector
###############################################
def combine_features(row):
    features = []
    # Add topic distribution (list of probabilities, e.g., 5 topics)
    features.extend(row['topic_dist'])
    # Add sentiment scores (polarity and subjectivity)
    features.append(row['polarity'])
    features.append(row['subjectivity'])
    # Add NER counts (for PERSON, ORG, GPE)
    features.append(row['person_count'])
    features.append(row['org_count'])
    features.append(row['gpe_count'])
    # Add domain-specific keyword counts (order by sorted key names)
    for key in sorted(domain_keywords.keys()):
        features.append(row.get(key, 0))
    return features

df2['feature_vector'] = df2.apply(combine_features, axis=1)

# Create a final feature matrix (each row is a document's feature vector)
X_multiclass = np.vstack(df2['feature_vector'].values)
y_multiclass = df2['label']

###############################################
# Example: Training a Multi-Class Classifier
###############################################
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_multiclass, y_multiclass)

# Evaluate performance (if you have a separate validation set, use that)
predictions = clf.predict(X_multiclass)
print("Classification Report for Multi-Class Classifier:\n", 
      pd.Series(predictions).value_counts(), "\n")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hazel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Classification Report on Test Set:
               precision    recall  f1-score   support

           0       0.73      0.71      0.72        79
           1       0.60      0.67      0.63        78
           2       0.86      0.80      0.82        83
           3       1.00      0.99      0.99        80

    accuracy                           0.79       320
   macro avg       0.80      0.79      0.79       320
weighted avg       0.80      0.79      0.79       320

Classification Report for Multi-Class Classifier:
 2    400
1    400
3    400
0    400
dtype: int64 



In [21]:
df2

content  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [22]:
from sklearn.metrics import classification_report
print("Full Classification Report:\n", classification_report(y_multiclass, predictions))

Full Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       400
           1       1.00      1.00      1.00       400
           2       1.00      1.00      1.00       400
           3       1.00      1.00      1.00       400

    accuracy                           1.00      1600
   macro avg       1.00      1.00      1.00      1600
weighted avg       1.00      1.00      1.00      1600

